## Goal
* Gather a table of sunset and sunrise times from an array of locations and store results

https://sunrise-sunset.org/api

In [4]:
# Import tools
from urllib.request import urlopen
from datetime import timedelta, date, datetime
from dateutil import tz
import json
import pandas as pd

## Build Date Array

In [5]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Initialize date range
start_date = date(2017, 7, 1)
end_date = date(2017, 11, 1)

dates = []

for single_date in daterange(start_date, end_date):
        dates.append(single_date.strftime("%Y-%m-%d"))

In [6]:
# display sample
for date in dates[-4:]:
    print (date)

2017-10-28
2017-10-29
2017-10-30
2017-10-31


## Load Location Dictionary

In [7]:
# static location selection
location = 'missoula'
lat = '46.872'
lon = '113.9940'

In [8]:
# load location df
stations = pd.read_csv('../output/weather_monitoring-locations.csv')

In [9]:
# display sample
stations.loc[:1]

,id,lat,lon,neighborhood,city
0,KMTCORVA9,46.277557,-114.045059,Hamilton Heights,Corvallis
1,KMTCORVA10,46.284927,-114.106499,All View Lane,Corvallis


## Build URLs

In [31]:
# create empty data frame
df = pd.DataFrame(columns=['id','url'])

In [92]:
def build_url_df(stations, dates): 
    url_station = []
    # loop through stations in weather_monitoring-locations.csv
    for index, row in stations.iterrows():
        # loop through date range set above
        for date in dates:
            row_id = row['id']
            lat = row['lat']
            lon = row['lon']
            row_url = 'https://api.sunrise-sunset.org/json?lat=' + str(lat) + '&lng=' + str(lon) + '&date=' + date
            url_station.append(dict({'station_id': row_id, 'url': row_url, 'date': date}))
    return(url_station)

In [93]:
# generate url list for api calls
url_batch = build_url_df(stations, dates)

In [94]:
len(url_batch)

4182

## Call API and store results

In [104]:
sunset_sunrise = []

In [99]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    
    return parsed_json

#### Start with one call

In [105]:
# call and store one result
for url in url_batch[:1]:
    print(url['url'], url['station_id'])
    results = api_call(url['url'])
    # include station_id
    results['results']['station_id'] = url['station_id']
    # include date 
    results['results']['date'] = url['date']
    sunset_sunrise.append(results)
sunset_sunrise

https://api.sunrise-sunset.org/json?lat=46.277557&lng=-114.045059&date=2017-07-01 KMTCORVA9


[{'results': {'astronomical_twilight_begin': '9:06:37 AM',
   'astronomical_twilight_end': '6:13:53 AM',
   'civil_twilight_begin': '11:10:21 AM',
   'civil_twilight_end': '4:10:09 AM',
   'date': '2017-07-01',
   'day_length': '15:42:52',
   'nautical_twilight_begin': '10:18:31 AM',
   'nautical_twilight_end': '5:01:59 AM',
   'solar_noon': '7:40:15 PM',
   'station_id': 'KMTCORVA9',
   'sunrise': '11:48:49 AM',
   'sunset': '3:31:41 AM'},
  'status': 'OK'}]